# RFSoC QPSK Transceiver

> This design is a full QPSK transceiver, which transmits and receives randomly-generated pulse-shaped symbols with full carrier and timing synchronisation. PYNQ is used to visualise the data at both the DAC and ADC side of the RFSoC data converters, as well as visualising various DSP stages throughout the transmit and receive signal path.

 
## Contents    

* [RFSoC QPSK Transceiver](#RFSoC-QPSK-Transceiver)
    * [Import libraries](#Import-libraries)
    * [Download the QPSK bitstream](#Download-the-QPSK-bitstream)
    * [Inspecting the transmit path](#Inspecting-the-transmit-path)
    * [Inspecting the receive path](#Inspecting-the-receive-path)
    * [Reconfigure the RF Data Path](#Reconfigure-the-RF-Data-Path)

## Import libraries

Start by including the `xrfdc` drivers so we can configure the RF data converters, `ipywidgets` to make interactive controls, `numpy` for numerical analysis, and `rfsoc_qpsk` for the QPSK design.

In [ ]:
import xrfdc
import ipywidgets as ipw
import numpy as np

from rfsoc_qpsk.overlay import QpskOverlay

## Download the QPSK bitstream

In [ ]:
ol = QpskOverlay()

For a quick reference of all the things you can do with the QPSK overlay, ask the Python interpreter!
Pop open a new console (right click here and select "_New Console for Notebook_") and type `ol.plot_group?` to query a method of our new overlay. Tab completion works for discovery too.

## Inspecting the transmit path

There are 3 main steps in the QPSK transmit IP signal path:

1. Random symbol generation
2. Pulse shaping
3. Interpolation
  
This design "taps off" this path after the first two stages so we can inspect the signals in Jupyter Lab.
The RF data converter can be reconfigured from Python too - we'll look at that [later](#Reconfigure-the-RF-Data-Path).

![](./images/block_diagram/QPSK_system_block_diagrams_Tx_only.svg)

First we plot our raw QPSK symbols in the time domain.

In [ ]:
ol.plot_group(
  'tx_symbol',            # Plot group's ID
  ['time-binary'],        # List of plot types chosen from:
                          #   ['time','time-binary','frequency','constellation']
  ol.qpsk_tx.get_symbols, # Function to grab a buffer of samples
  500                     # Sample frequency (Hz)
)

We can stream new samples into this plot using the play/stop buttons. By default the samples are stored in a rolling buffer, so we can keep this running for a while without worrying too much about total memory usage. As you continue to work through this notebook though, you should stop any previous plot streams to keep your browser happy.

For the pulse shaped signal, let's have a look at the frequency domain too. This FFT is accelerated in the PL so we pass in an extra argument, `get_freq_data`, telling the plotting library how to grab the accelerated FFT data.

In [ ]:
ol.plot_group('tx_shaped', ['time', 'frequency'], ol.qpsk_tx.get_shaped_time, 4000,
              get_freq_data=ol.qpsk_tx.get_shaped_fft)

## Inspecting the receive path

The receive side is nearly the inverse of the transmit path (there's just some extra work for properly synchronising).

Again, there are taps off from a few places in the signal path:

1. After decimation
2. After coarse synchronisation
3. After root-raised-cosine filtering
4. and the data output

![](./images/block_diagram/QPSK_system_block_diagrams_Rx_only.svg)

Because there are a few different intermediate stages, let's reuse the same cells to plot any of them on-demand.

First we describe how to generate plots for each of the intermediate taps.

In [ ]:
rx_domains = ['time', 'frequency', 'constellation']

plot_rx_decimated   = lambda : ol.plot_group(
    'rx_decimated',   rx_domains, ol.qpsk_rx.get_decimated,     4000
)

plot_rx_coarse_sync = lambda : ol.plot_group(
    'rx_coarse_sync', rx_domains, ol.qpsk_rx.get_coarse_synced, 4000
)

plot_rx_rrced       = lambda : ol.plot_group(
    'rx_rrced',       rx_domains, ol.qpsk_rx.get_rrced,         16000
)

Now we can just execute the function whichever tap you want. For example, let's look at the tap after decimation below.

In [ ]:
plot_rx_decimated()

And for the final plot, let's look at the synchronised output data. To recover the bits we need to take our sampled, synchronised signal (seen in the constellation plot below) and decide which quadrant each symbol has fallen into.

In [ ]:
def classify_bits(frame):
    bit_quantise    = lambda b: 1 if b>0 else 0
    symbol_quantise = lambda i, q: bit_quantise(i) + 1j*bit_quantise(q)
    return np.fromiter(
        map(symbol_quantise, np.real(frame), np.imag(frame)),
        dtype=np.complex
    )

ol.plot_group(
    'rx_data',
    ['constellation', 'time-binary'],
    lambda : classify_bits(ol.qpsk_rx.get_data()),
    500,
    get_const_data=ol.qpsk_rx.get_data
)

Now is a good time to note that Jupyter Lab can manage multiple windows. Next we'll be playing with the RF settings, so you may want to make a new window for the constellation plot and leave it streaming. Make a new window for the plot by right clicking the plot and selecting "_Create New View for Output_". Feel free to snap this new window to the side by clicking the window's title ("Output View") and dragging it to the side of the web page. Now we can play with RF settings further down the notebook while still getting instant feedback about our received signal — pretty neat!

## Reconfigure the RF Data Path

### Transmit Power

The QPSK bitstream includes a digital attenuator on the transmit path. We can configure this via a memory-mapped register.

Let's use this as an example of interactive reconfiguration because the effects are quite clear in the constellation diagram. Try reducing the output power by setting a gain between 0 (off) and 1 (full scale).

In [ ]:
ol.qpsk_tx.set_gain(0.6)

The constellation plot should shrink in a little towards the origin. Let's return to full power now.

In [ ]:
ol.qpsk_tx.set_gain(1)

We can use some `ipywidgets` to make a more natural interface to control the gain too. Let's expose this as a slider with a callback to the `set_gain` function.

In [ ]:
pow_slider = ipw.SelectionSlider(
    options=[0.1, 0.3, 0.6, 1],
    value=1,
    description='',
)

accordion = ipw.Accordion(children=[pow_slider])
accordion.set_title(0, 'Transmitter power')
display(accordion)

def unwrap_slider_val(callback):
    return lambda slider_val : callback(slider_val['new'])

pow_slider.observe(unwrap_slider_val(ol.qpsk_tx.set_gain), names='value')

### Transmit and Receive Mixer Settings

So far the RF Data Converter settings have been controlled by `QpskOverlay` but we can also reconfigure these on the fly in python with the `xrfdc` driver.

First of all, consider the DAC block used for the transmit side.

![](./images/block_diagram/RF_DAC.svg)

There's a lot of scope for reconfiguration here — see the [IP product guide](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_1/pg269-rf-data-converter.pdf) or type `ol.dac_block?` for more details.

As an example, let's play with the mixer settings. Try changing the DAC's mixer frequency from the deafult 1000 MHz to 900 MHz.

In [ ]:
def update_nco(rf_block, nco_freq):
    mixer_cfg = rf_block.MixerSettings
    mixer_cfg['Freq'] = nco_freq
    rf_block.MixerSettings = mixer_cfg
    rf_block.UpdateEvent(xrfdc.EVENT_MIXER)

update_nco(ol.dac_block, 900)

The received signal should disappear until we configure the receiver's ADC to match the new carrier frequency. Set the new carrier frequency for the ADC side mixer below.

In [ ]:
update_nco(ol.adc_block, 900)

Again, we can use `ipywidgets` to make an interactive interface for these settings. Below we setup an RX and a TX slider and a TX slider that are linked together so we can scrub along the spectrum keeping both sides in near lock-step. If you've got any analog RF filters to hand, try them out with different mixer settings!

In [ ]:
def new_nco_slider(title):
    return ipw.FloatSlider(
        value=1000,
        min=620,
        max=1220,
        step=20,
        description=title,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )

tx_nco_slider = new_nco_slider('TX (MHz)')
rx_nco_slider = new_nco_slider('RX (MHz)')

accordion = ipw.Accordion(children=[ipw.VBox([tx_nco_slider, rx_nco_slider])])
accordion.set_title(0, 'Carrier frequency')
display(accordion)

ipw.link((rx_nco_slider, 'value'), (tx_nco_slider, 'value'))
tx_nco_slider.observe(
    unwrap_slider_val(lambda v: update_nco(ol.dac_block, v)),
    names='value'
)
rx_nco_slider.observe(
    unwrap_slider_val(lambda v: update_nco(ol.adc_block, v)),
    names='value'
)

## Summary

We've now lead you through how we can interact with the RF data converters from PYNQ, using a QPSK transmit/receive loopback system as an example. More exhaustively, we've shown:

  * Use of the programmable logic in the context of a real RF application
  * Performing on-board introspection of an RF design:
      * Leveraging existing plotting libraries from the Python ecosystem
  * Interacting with a QPSK hardware design
      * Configuring the signal path, using transmit power as an example
      * Configuring the RF data converter, using TX/RX mixer frequencies as an example